# FIT5148 Distributed Database and Big Data 
## Assignement 1
Student1  Name : Rui Liang
Student1 Number: 27735893

Student2 Name: Xitong Jiang
Student2 Number: 28402987


### Task1.1 Parallel Search
ClimateData consists of 12 months.
By using hash partition, every partition only have the data come from the same month
After partitioning by month, we can only search one partition which contains the same month as the given key.
Linear search is more suitable in this task for there is less than 31 days in a month

In [1]:
import csv
FireData = open("FireData.csv","r")
reader = csv.reader(FireData)
FireData=[]
for item in reader:
    FireData.append(item)
FireData= FireData[1:]

ClimateData = open("ClimateData.csv","r")
reader = csv.reader(ClimateData)
ClimateData=[]
for item in reader:
    ClimateData.append(item)
ClimateData= ClimateData[1:]

In [2]:
# Linear search function for task1_1
def linear_search_task1_1(data,key):
    matched_record=None
    for x in data:
        if x[1]==key:
            matched_record =x
            break
    return  matched_record

In [3]:
def s_hash(date,t):
    result=int(date.month)%t
    return result

In [4]:
#Hash data partionining function for task1_1
import datetime
def h_partition_task1_1(data,n):
    dic ={}
    for entry in data:
        entry = tuple(entry)# list cannot update or add in set
        date=datetime.datetime.strptime(entry[1],'%Y-%m-%d')
        
        h = s_hash(date,n)
        if (h in dic.keys()):
            s=dic[h]
            s.add(entry)
            dic[h]=s
        else:
            s=set()
            s.update(entry)
            dic[h]=s
    return dic

In [5]:
def Parallel_search_task1_1(data, n_processor,key):
    
    date=datetime.datetime.strptime(key,'%Y-%m-%d')
    
    a = int(date.month)%12
    
    Climate_month=h_partition_task1_1(ClimateData,n_processor)[a]
    
    Climate_month=list(Climate_month)
    
    result=linear_search_task1_1(Climate_month,key)
    
    return result

In [6]:
Parallel_search_task1_1(ClimateData, 12,'2017-12-15')

('948702', '2017-12-15', '18', '52', '7.1', '14', '   74.5*', '53.1', ' 0.00I')

### Task 1.2
we need to search surface temperature in a range specific range. 
After partition by range, the method will search those partition which contains the search surface temperature.
For lots of repeated data in surface temperature such as 65,linear search is used to scan every item in the processor and find the matched repeated record.

In [7]:
def range_partition_T1_2(data, range_indices):
    result = []
    data.sort(key = lambda x:int(x[7])) 
    n_bin = len(range_indices)
    for i in range(n_bin):
        s = [x for x in data if int(x[7]) < range_indices[i]]
        result.append(s)
        last_element = s[len(s)-1]
        last = data.index(last_element)
        data = data[int(last)+1:]
    result.append([x for x in data if int(x[7]) >= range_indices[n_bin-1]])
    return result

In [8]:
def linear_search_T1_2(d,key):
    result=[]
    for x in d:
        if(int(x[7])==key):
            result.append(x) 
    return result

In [9]:
import multiprocessing as mp
def parallel_search_range_T1_2(data, query_range, n_processor):
    results = []
    pool = mp.Pool(processes = n_processor)
    DD = range_partition_T1_2(data, [48,88])
    #print(DD)
    minl = query_range[0]
    maxl = query_range[1]
    for d in DD :
        last_index = len(d)-1
        if int(d[last_index][7]) >= minl and int(d[0][7]) <= maxl:
            for i in range(minl,maxl):
                result = pool.apply(linear_search_T1_2, [d,i])  
                if result != None:
                    for i in result:
                        results.append({",".join([i[0],i[1],i[5],i[7]])})

    return results

In [10]:
parallel_search_range_T1_2(FireData, [65,100], 3)

[{'-37.642,149.263,100,65'},
 {'-37.862,144.175,87,65'},
 {'-37.331,143.122,90,65'},
 {'-37.46,148.102,88,65'},
 {'-37.446,148.102,100,65'},
 {'-37.396,148.086,100,65'},
 {'-38.3998,147.064,89,65'},
 {'-38.4792,146.3081,89,65'},
 {'-36.5661,142.2956,89,65'},
 {'-35.4666,142.0749,89,65'},
 {'-36.7218,141.6411,89,65'},
 {'-36.4143,143.272,89,65'},
 {'-37.876,143.7804,89,65'},
 {'-36.2618,141.8783,89,65'},
 {'-37.0959,143.8206,89,65'},
 {'-34.9023,142.0557,89,65'},
 {'-36.73,143.935,89,65'},
 {'-36.0295,143.6409,89,65'},
 {'-38.1561,143.7846,89,65'},
 {'-35.7642,143.3321,89,65'},
 {'-36.1441,145.2221,89,65'},
 {'-37.88,143.7233,89,65'},
 {'-37.5816,148.5862,84,65'},
 {'-37.8062,143.3598,77,65'},
 {'-36.9902,141.879,89,65'},
 {'-37.745,142.993,89,65'},
 {'-34.3943,141.7567,89,65'},
 {'-37.7746,148.3673,88,65'},
 {'-37.4471,147.6331,89,65'},
 {'-37.4449,147.6594,89,65'},
 {'-35.4255,143.6306,89,65'},
 {'-36.1556,141.5904,89,65'},
 {'-35.7647,142.0858,89,65'},
 {'-37.8973,143.4754,82,65'},
 

### Task2.1 Parallel Join
ClimateData consists of 12 months. 
By using hash partition, every partition only have the data come from the same month.
After partitioning by month, we can join the ClimateData and FireData in same month. 

In [11]:
def NL_join(T1,T2):
    result = []

    for tr1 in T1:

        for tr2 in T2:
            if (tr1[6] == tr2[1]):
                result.append({", ".join([tr1[3], tr1[7], tr2[2],tr2[3],tr2[5]])}) 
    return result

In [12]:
def s_hash_Fire(entry,n):
    date=datetime.datetime.strptime(entry[6],'%Y-%m-%d')
    result=int(date.month)%n
    return result

In [13]:
def s_hash_Climate(entry,n):
    date=datetime.datetime.strptime(entry[1],'%Y-%m-%d')
    result=int(date.month)%n
    return result

In [14]:
import datetime
def h_partition_task1_1_Fire(data,n):
    data = sorted(data,key=lambda x: x[3])
    dic ={}
    for entry in data:
        entry = tuple(entry)# list cannot update or add in set
        h = s_hash_Fire(entry,n)
        if (h in dic.keys()):
            s=dic[h]
            s.add(entry)
            dic[h]=s
        else:
            s=set()
            s.update({entry})
            dic[h]=s
    return dic

In [15]:
import datetime
def h_partition_task1_1_Climate(data,n):
    data = sorted(data,key=lambda x: x[1])
    dic ={}
    for entry in data:
        entry = tuple(entry)# list cannot update or add in set
        h = s_hash_Climate(entry,n)
        if (h in dic.keys()):
            s=dic[h]
            s.add(entry)
            dic[h]=s
        else:
            s=set()
            s.update({entry})
            dic[h]=s
    return dic

In [16]:
import multiprocessing as mp
def join_task2_1(T1, T2, n_processor):
   
    results = ["Surface temperature(C),air temperature(C),realtive humidity,maximum wind speed"]
 
    T1_subsets = h_partition_task1_1_Climate(T1, n_processor)
    T2_subsets = h_partition_task1_1_Fire(T2, n_processor)
    a=list(T1_subsets.keys())
    b=list(T2_subsets.keys())
    result_key=[]
    for i in a[:]:
        for j in b[:]:
            if (i == j):
                result_key.append(i) 
    pool = mp.Pool(processes = n_processor)
    result_key=list(result_key)
    for i in result_key :
        item1=T1_subsets[i]
        item2= T2_subsets[i]
        result=pool.apply(NL_join,[item2, item1])
        results.append(result)
        print(result)
    return results


In [17]:
join_task2_1(ClimateData, FireData,12)

[{'2017-12-16T15:38:39, 43, 18, 53.7, 13'}, {'2017-12-16T04:34:58, 75, 18, 53.7, 13'}, {'2017-12-16T04:34:57, 56, 18, 53.7, 13'}, {'2017-12-16T04:34:57, 55, 18, 53.7, 13'}, {'2017-12-15T13:17:17, 38, 18, 52, 14'}, {'2017-12-25T04:29:08, 54, 17, 50.4, 16.9'}, {'2017-12-21T04:53:29, 46, 17, 52.5, 11.1'}, {'2017-12-15T13:17:17, 42, 18, 52, 14'}, {'2017-12-08T13:11:44, 54, 15, 48.1, 12'}, {'2017-12-16T15:38:39, 33, 18, 53.7, 13'}, {'2017-12-27T00:02:15, 53, 28, 58.3, 15.9'}, {'2017-12-16T04:34:57, 60, 18, 53.7, 13'}, {'2017-12-16T04:35:13, 73, 18, 53.7, 13'}, {'2017-12-16T04:34:57, 66, 18, 53.7, 13'}, {'2017-12-27T04:16:51, 68, 28, 58.3, 15.9'}, {'2017-12-08T13:11:43, 52, 15, 48.1, 12'}, {'2017-12-16T00:21:11, 48, 18, 53.7, 13'}, {'2017-12-16T04:35:15, 54, 18, 53.7, 13'}, {'2017-12-10T00:57:59, 38, 17, 53.5, 14'}, {'2017-12-16T00:20:53, 57, 18, 53.7, 13'}, {'2017-12-16T00:21:12, 73, 18, 53.7, 13'}, {'2017-12-15T13:17:17, 36, 18, 52, 14'}, {'2017-12-08T13:11:43, 40, 15, 48.1, 12'}, {'2017-1

['Surface temperature(C),air temperature(C),realtive humidity,maximum wind speed',
 [{'2017-12-16T15:38:39, 43, 18, 53.7, 13'},
  {'2017-12-16T04:34:58, 75, 18, 53.7, 13'},
  {'2017-12-16T04:34:57, 56, 18, 53.7, 13'},
  {'2017-12-16T04:34:57, 55, 18, 53.7, 13'},
  {'2017-12-15T13:17:17, 38, 18, 52, 14'},
  {'2017-12-25T04:29:08, 54, 17, 50.4, 16.9'},
  {'2017-12-21T04:53:29, 46, 17, 52.5, 11.1'},
  {'2017-12-15T13:17:17, 42, 18, 52, 14'},
  {'2017-12-08T13:11:44, 54, 15, 48.1, 12'},
  {'2017-12-16T15:38:39, 33, 18, 53.7, 13'},
  {'2017-12-27T00:02:15, 53, 28, 58.3, 15.9'},
  {'2017-12-16T04:34:57, 60, 18, 53.7, 13'},
  {'2017-12-16T04:35:13, 73, 18, 53.7, 13'},
  {'2017-12-16T04:34:57, 66, 18, 53.7, 13'},
  {'2017-12-27T04:16:51, 68, 28, 58.3, 15.9'},
  {'2017-12-08T13:11:43, 52, 15, 48.1, 12'},
  {'2017-12-16T00:21:11, 48, 18, 53.7, 13'},
  {'2017-12-16T04:35:15, 54, 18, 53.7, 13'},
  {'2017-12-10T00:57:59, 38, 17, 53.5, 14'},
  {'2017-12-16T00:20:53, 57, 18, 53.7, 13'},
  {'2017-12-1

### Task 2.2
An algotithm to find datetime, air temperature, surface temperature and confidence when the confidence is between 80 and 100. In this task, because confidence should be between 80 and 100, so range partition is a better way to partitioning dataset. Then do join operation between matched data and climatedata file.

In [18]:
def range_partition(data,range_indices,index):
    result = []
    new_data = list(data)
    
    new_data.sort(key = lambda x : int(x[index]))
    n_bin = len(range_indices)
    for i in range(n_bin):
        s = [x for x in new_data if int(x[index]) < range_indices[i]]
        result.append(s)
        last_element = s[len(s) - 1]
        last = new_data.index(last_element)
        new_data = new_data[int(last) + 1:]
    result.append([x for x in new_data if int(x[index]) >= range_indices[n_bin - 1]])
    return result

In [19]:
def NL_join_task2(T1,T2):
    result = []

    for tr2 in T2:

        for tr1 in T1:
            if (tr1[6] == tr2[1]):
                result.append({", ".join([str(tr1[3]),str(tr2[2]),str(tr1[7]),str(tr1[5])])}) 
    return result

In [20]:
NL_join_task2(range_partition(FireData,[80,100],5)[1],ClimateData)

[{'2017-03-06T05:06:20, 20, 59, 85'},
 {'2017-03-06T05:06:30, 20, 62, 87'},
 {'2017-03-07T04:16:10, 19, 64, 88'},
 {'2017-03-09T13:23:40, 23, 41, 86'},
 {'2017-03-10T04:43:50, 19, 55, 81'},
 {'2017-03-10T04:45:30, 19, 73, 94'},
 {'2017-03-10T04:45:30, 19, 80, 97'},
 {'2017-03-12T04:28:00, 21, 68, 80'},
 {'2017-03-12T04:29:50, 21, 71, 84'},
 {'2017-03-12T04:27:20, 21, 98, 85'},
 {'2017-03-12T04:33:50, 21, 105, 94'},
 {'2017-03-14T04:14:40, 25, 55, 81'},
 {'2017-03-14T04:16:20, 25, 65, 89'},
 {'2017-03-14T04:24:40, 25, 67, 90'},
 {'2017-03-14T04:20:50, 25, 68, 90'},
 {'2017-03-14T04:22:20, 25, 72, 93'},
 {'2017-03-14T04:14:30, 25, 75, 94'},
 {'2017-03-14T04:16:30, 25, 75, 95'},
 {'2017-03-14T04:14:30, 25, 79, 96'},
 {'2017-03-17T04:45:10, 18, 59, 84'},
 {'2017-03-17T04:45:00, 18, 60, 86'},
 {'2017-03-17T04:48:20, 18, 74, 94'},
 {'2017-03-18T03:50:40, 21, 80, 97'},
 {'2017-03-19T04:32:50, 24, 58, 84'},
 {'2017-03-19T04:39:50, 24, 60, 85'},
 {'2017-03-19T04:36:00, 24, 66, 90'},
 {'2017-03-

In [21]:
from multiprocessing import Pool
def join_task2_2(T1,T2,n_processor,range_indices,index):# Perform a disjoint partitioning-based parallel join algorithm.

    result = ["Datetime,Air Temperature,Surface Temperature,Confidence "]
    
    T1_subsets = range_partition(T1,range_indices,index)

    pool = Pool(processes = n_processor)
    
    result.append(pool.apply(NL_join_task2,[T1_subsets[1],T2]))
    
    return result

In [22]:
join_task2_2(FireData, ClimateData,3,[80,100],5)

['Datetime,Air Temperature,Surface Temperature,Confidence ',
 [{'2017-03-06T05:06:20, 20, 59, 85'},
  {'2017-03-06T05:06:30, 20, 62, 87'},
  {'2017-03-07T04:16:10, 19, 64, 88'},
  {'2017-03-09T13:23:40, 23, 41, 86'},
  {'2017-03-10T04:43:50, 19, 55, 81'},
  {'2017-03-10T04:45:30, 19, 73, 94'},
  {'2017-03-10T04:45:30, 19, 80, 97'},
  {'2017-03-12T04:28:00, 21, 68, 80'},
  {'2017-03-12T04:29:50, 21, 71, 84'},
  {'2017-03-12T04:27:20, 21, 98, 85'},
  {'2017-03-12T04:33:50, 21, 105, 94'},
  {'2017-03-14T04:14:40, 25, 55, 81'},
  {'2017-03-14T04:16:20, 25, 65, 89'},
  {'2017-03-14T04:24:40, 25, 67, 90'},
  {'2017-03-14T04:20:50, 25, 68, 90'},
  {'2017-03-14T04:22:20, 25, 72, 93'},
  {'2017-03-14T04:14:30, 25, 75, 94'},
  {'2017-03-14T04:16:30, 25, 75, 95'},
  {'2017-03-14T04:14:30, 25, 79, 96'},
  {'2017-03-17T04:45:10, 18, 59, 84'},
  {'2017-03-17T04:45:00, 18, 60, 86'},
  {'2017-03-17T04:48:20, 18, 74, 94'},
  {'2017-03-18T03:50:40, 21, 80, 97'},
  {'2017-03-19T04:32:50, 24, 58, 84'},
  

### Task 3 Parallel Sort
First, round-robin partition is used to break record into sub-records serial sorting is used in each sub-records to sorted the data Binary merging is used to merge the partition in to final result

In [23]:
def qsort(arr): 
    for item in arr:
        item[-1]=int(item[-1])
    if len(arr) <= 1:
        return arr
    else:
        return qsort([x for x in arr[1:] if x[7] < arr[0][7]])\
                + [arr[0]] \
                + qsort([x for x in arr[1:] if x[7] >= arr[0][7]])

In [24]:
# Round-robin data partitionining function
def rr_partition(data, n):
    data = qsort(data)
    result = []
    for i in range(n):
        result.append([])
    n_bin = len(data)/n

    for index, element in enumerate(data): 
        index_bin = (int) (index % n)
        result[index_bin].append(element)
    
    return result

In [25]:
import sys

def find_min(records):    
    """ 
    Find the smallest record
    
    Arguments:
    records -- the input record set

    Return:
    result -- the smallest record's index
    """
    m = int(records[0][-1]) ##Using -1 solves index out of bound exception
    index = 0
    for i in range(len(records)):
        a =int(records[i][-1]) ##Using -1 solves index out of bound exception
        if(a < m):  
            index = i
            m = int(records[i][7])
    return index

def k_way_merge(record_sets):    
    # indexes will keep the indexes of sorted records in the given buffers
    indexes = []
    for x in record_sets:
        indexes.append(0) # initialisation with 0

    result = []  
    tuple = []
    
    while(True):
        tuple = [] # initialise tuple
    
        # This loop gets the current position of every buffer
        for i in range(len(record_sets)):
           
            if(indexes[i] >= len(record_sets[i])):              
                tuple.append([sys.maxsize])       
               
            else:
                tuple.append(record_sets[i][indexes[i]]) 
                
        smallest = find_min(tuple)
        if(tuple[smallest][-1] == sys.maxsize):
            break

        # This record is the next on the merged list
        result.append(record_sets[smallest][indexes[smallest]])

        indexes[smallest] +=1
    return result

In [26]:
def serial_sorting(dataset, buffer_size):
    
    if (buffer_size <= 2):
        print("Error: buffer size should be greater than 2")
        return
    
    result = []
    
    # --- Sort Phase ---
    sorted_set = []
    start_pos = 0
    N = len(dataset) 
    while True:
        if ((N - start_pos) > buffer_size):
            subset = dataset[start_pos:start_pos + buffer_size] 
            sorted_subset = qsort(subset) 
            sorted_set.append(sorted_subset)
            start_pos += buffer_size
        else:
            subset = dataset[start_pos:] 
            sorted_subset = qsort(subset) 
            sorted_set.append(sorted_subset)
            break

    # --- Merge Phase ---
    merge_buffer_size = buffer_size - 1 # /3
    dataset = sorted_set
    while True:
        merged_set = []
        N = len(dataset)
        start_pos = 0
        while True:
            if ((N - start_pos) > merge_buffer_size): 
                subset = dataset[start_pos:start_pos + merge_buffer_size]

                merged_set.append(k_way_merge(subset)) # merge lists in subset
                start_pos += merge_buffer_size

            else:
                subset = dataset[start_pos:]
                merged_set.append(k_way_merge(subset)) # merge lists in subset
                break

        dataset = merged_set
        if (len(dataset) <= 1): # if the size of merged record set is 1, then stop 
            result = merged_set
            break
    
    return result

In [27]:
import multiprocessing as mp

def Task3_parallel_binary_merge_sorting(dataset, n_processor, buffer_size):

    
    if (buffer_size <= 2):
        print("Error: buffer size should be greater than 2")
        return
    
    result = []
    subsets = rr_partition(dataset,n_processor)
 
    pool = mp.Pool(processes = n_processor)

    # ----- Sort phase -----
    sorted_set = []
    for s in subsets:
        # call the serial_sorting method above
        sorted_set.append(*pool.apply(serial_sorting, [s, buffer_size]))
    pool.close()
    
    # ---- Final merge phase ----

    result1=sorted_set
    while len(result1)!=1:
        for i in range(0,len(result1),2):
            result+=[k_way_merge([result1[i]]+[result1[i+1]])]
        result1=result
        result=[]
        
    return result1

In [28]:
Task3_parallel_binary_merge_sorting(FireData,4,10)

[[['-37.886',
   '147.207',
   '302',
   '2017-07-02T04:28:42',
   '10.7',
   '50',
   '2017-07-02',
   28],
  ['-37.886',
   '147.207',
   '302',
   '2017-07-02T04:28:42',
   '10.7',
   '50',
   '2017-07-02',
   28],
  ['-37.062',
   '141.373',
   '303.1',
   '2017-07-01T13:11:41',
   '16.1',
   '53',
   '2017-07-01',
   29],
  ['-37.062',
   '141.373',
   '303.1',
   '2017-07-01T13:11:41',
   '16.1',
   '53',
   '2017-07-01',
   29],
  ['-36.943',
   '143.286',
   '302.7',
   '2017-11-11T15:08:00',
   '18.8',
   '51',
   '2017-11-11',
   29],
  ['-37.466',
   '148.1',
   '302.2',
   '2017-10-02T23:44:31',
   '10.9',
   '50',
   '2017-10-02',
   29],
  ['-37.38',
   '149.334',
   '304.5',
   '2017-11-30T15:38:32',
   '14.1',
   '61',
   '2017-11-30',
   31],
  ['-37.227',
   '141.146',
   '305.1',
   '2017-10-03T01:22:44',
   '41.2',
   '54',
   '2017-10-03',
   31],
  ['-35.646',
   '142.282',
   '305.6',
   '2017-12-24T13:12:01',
   '11.8',
   '65',
   '2017-12-24',
   32],
  ['-37.

### Task 4.1 Parallel Group-By
Get the total number of fire in each day and display total number of fire and the date.
In this task, using hash partition to partitioning data with 12 months, each hash value holds the data with same month. Then each processor does parallel local groupby operation to get the total number of fire. After getting the result from each processor, doing global groupby operation aggregates all processors.

In [29]:
import datetime
def s_hash_Fire(entry,n):
    date=datetime.datetime.strptime(entry[6],'%Y-%m-%d')
    result=int(date.month)%n
    return result

In [30]:
# Partition by month ,there is 12 different month in the data
import datetime
def h_partition_task1_1_Fire(data,n):
    data = sorted(data,key=lambda x: x[3])
    dic ={}
    for entry in data:
        entry = tuple(entry)# list cannot update or add in set
        h = s_hash_Fire(entry,n)
        if (h in dic.keys()):
            s=dic[h]
            s.add(entry)
            dic[h]=s
        else:
            s=set()
            s.update({entry})
            dic[h]=s
    return dic

In [31]:
# The first step in the merge-all groupby method
def local_groupby_task4_1(dataset):

    dict = {}
    for index, record in enumerate(dataset):
        key = record[6]
        val = 1
        if key not in dict:
            dict[key] = 0
        dict[key] += val
    return dict

In [32]:
import multiprocessing as mp

def parallel_groupby_task4_1(dataset):

    result = {}

    n_processor = len(dataset)
    pool = mp.Pool(processes = n_processor)

    # ----- Local aggregation step -----
    local_result = []
    for s in dataset:
        
        local_result.append(pool.apply(local_groupby_task4_1, [dataset[s]]))
        
    pool.close()

    # ---- Global aggregation step ----
    
    result = local_result[0]
    for i in range(1,len(local_result)):
        for key in result.keys():
            if key in local_result[i].keys():
                result[key] = result[key] + local_result[i][key]
        for key in local_result[i].keys():
            if key not in result.keys():
                result[key] = local_result[i][key]
    
    return result

In [33]:
parallel_groupby_task4_1(h_partition_task1_1_Fire(FireData,12))

{'2017-03-06': 2,
 '2017-03-07': 1,
 '2017-03-08': 2,
 '2017-03-09': 3,
 '2017-03-10': 8,
 '2017-03-12': 5,
 '2017-03-13': 2,
 '2017-03-14': 10,
 '2017-03-15': 7,
 '2017-03-17': 6,
 '2017-03-18': 3,
 '2017-03-19': 21,
 '2017-03-24': 2,
 '2017-03-25': 13,
 '2017-03-26': 17,
 '2017-03-28': 54,
 '2017-03-29': 1,
 '2017-03-31': 22,
 '2017-04-01': 7,
 '2017-04-02': 5,
 '2017-04-03': 72,
 '2017-04-04': 89,
 '2017-04-05': 49,
 '2017-04-06': 118,
 '2017-04-07': 39,
 '2017-04-08': 20,
 '2017-04-11': 24,
 '2017-04-12': 69,
 '2017-04-13': 357,
 '2017-04-14': 18,
 '2017-04-15': 69,
 '2017-04-16': 18,
 '2017-04-17': 38,
 '2017-04-18': 325,
 '2017-04-19': 50,
 '2017-04-20': 31,
 '2017-04-22': 2,
 '2017-04-23': 19,
 '2017-04-24': 8,
 '2017-04-25': 3,
 '2017-04-26': 1,
 '2017-04-29': 3,
 '2017-05-01': 20,
 '2017-05-02': 10,
 '2017-05-03': 64,
 '2017-05-04': 135,
 '2017-05-05': 31,
 '2017-05-06': 17,
 '2017-05-07': 3,
 '2017-05-08': 24,
 '2017-05-09': 13,
 '2017-05-10': 114,
 '2017-05-11': 19,
 '2017-0

### Task4.2 
Find the average surface temperature for each day, and display average surface temperature and the date.
In this task, using hash partition to partitioning data with 12 months, each hash value holds the data with same month. Then each processor does parallel local groupby operation. After getting the result of each processor, doing global groupby operation aggregates all processors.

In [34]:
import datetime
def s_hash(entry,n):
    date=datetime.datetime.strptime(entry[6],'%Y-%m-%d')
    result=int(date.month)%n
    return result

In [35]:
# Partition by month ,there is 12 different month in the data
import datetime
def h_partition(data,n):
    data = sorted(data,key=lambda x: x[3])
    dic ={}
    for entry in data:
        entry = tuple(entry)# list cannot update or add in set
        h = s_hash(entry,n)
        if (h in dic.keys()):
            s=dic[h]
            s.add(entry)
            dic[h]=s
        else:
            s=set()
            s.update({entry})
            dic[h]=s
    return dic

In [36]:
def local_groupby(dataset):
    dict = {}
    for index, record in enumerate(dataset):
        key = record[6]
        val = int(record[7])
        if key not in dict:
            dict[key] = [0,0]
        dict[key][0] += int(val)
        dict[key][1] += 1
    return dict

In [37]:
import multiprocessing as mp

def parallel_merge_all_groupby_task4_2(dataset,n_processor):

    results = {}
    sub_dataset = h_partition(dataset,n_processor)
    pool = mp.Pool(processes = n_processor)
    local_result = []
    # local groupby
    for s in sub_dataset:

        local_result.append(pool.apply(local_groupby, [sub_dataset[s]]))
    pool.close()
    
    # global groupby, for each processor aggregate the result
    for i in local_result:
        for j in i:
            if j not in results:
                results[j] = i[j]
            else:
                results[j][0] += i[j][0]
                results[j][1] += i[j][1]
    for i in results:
        results[i] = int(results[i][0]/results[i][1])
    
    return results

In [38]:
parallel_merge_all_groupby_task4_2(FireData,12)

{'2017-03-06': 60,
 '2017-03-07': 64,
 '2017-03-08': 51,
 '2017-03-09': 46,
 '2017-03-10': 69,
 '2017-03-12': 88,
 '2017-03-13': 38,
 '2017-03-14': 65,
 '2017-03-15': 46,
 '2017-03-17': 59,
 '2017-03-18': 79,
 '2017-03-19': 65,
 '2017-03-24': 49,
 '2017-03-25': 66,
 '2017-03-26': 56,
 '2017-03-28': 60,
 '2017-03-29': 51,
 '2017-03-31': 48,
 '2017-04-01': 46,
 '2017-04-02': 45,
 '2017-04-03': 58,
 '2017-04-04': 62,
 '2017-04-05': 53,
 '2017-04-06': 61,
 '2017-04-07': 50,
 '2017-04-08': 60,
 '2017-04-11': 46,
 '2017-04-12': 52,
 '2017-04-13': 58,
 '2017-04-14': 61,
 '2017-04-15': 59,
 '2017-04-16': 48,
 '2017-04-17': 50,
 '2017-04-18': 53,
 '2017-04-19': 54,
 '2017-04-20': 56,
 '2017-04-22': 54,
 '2017-04-23': 53,
 '2017-04-24': 59,
 '2017-04-25': 48,
 '2017-04-26': 34,
 '2017-04-29': 63,
 '2017-05-01': 68,
 '2017-05-02': 55,
 '2017-05-03': 56,
 '2017-05-04': 56,
 '2017-05-05': 51,
 '2017-05-06': 57,
 '2017-05-07': 50,
 '2017-05-08': 56,
 '2017-05-09': 42,
 '2017-05-10': 52,
 '2017-05-11

## Task 5

In [39]:
def join(T1,T2):
    
    result = []
    
    for tr1 in T1:
    
        for tr2 in T2:
        
            if (tr1[6] == tr2[1]):

                result.append([tr1[7], str(tr2[1]), tr2[0]])
              
    return result

In [40]:
import datetime
def s_hash(entry,n):
    
    date=datetime.datetime.strptime(entry[1],'%Y-%m-%d')
    
    result=int(date.month)%n
    
    return result

In [41]:
import datetime
def h_partition(data,n):
    data = sorted(data,key=lambda x: x[1])
    dic ={}
    for entry in data:
        entry = tuple(entry)# list cannot update or add in set
        h = s_hash(entry,n)
        if (h in dic.keys()):
            s=dic[h]
            s.append(entry)
            dic[h]=s
        else:
            s=list()
            s.append(entry)
            dic[h]=s
    return dic

In [42]:
def local_groupby(dataset):
    
    dict = {}
    for index, record in enumerate(dataset):
        key = record[2]
        val = int(record[0])
        
        if key not in dict:
            dict[key] = [0,0]
        dict[key][0] += int(val)
        dict[key][1] += 1
        
    return dict

In [43]:
import multiprocessing as mp

def parallel_merge_all_groupby_task5(dataset,n_processor):

    results = {}
    sub_dataset = h_partition(dataset,n_processor)
    pool = mp.Pool(processes = n_processor)
    local_result = []

    for s in sub_dataset:
        
        local_result.append(pool.apply(local_groupby, [sub_dataset[s]]))
  
    pool.close()

    # global groupby
    for i in local_result:
        for j in i:
            if j not in results:
                results[j] = i[j]
            else:
                results[j][0] += i[j][0]
                results[j][1] += i[j][1]

    for i in results:
        results[i] = '%.2f' % float(results[i][0]/results[i][1])
    
    return results

In [44]:
parallel_merge_all_groupby_task5(join(FireData,ClimateData),12)

{'948701': '56.07', '948702': '52.15'}